In [ ]:
import ROOT
import os
%jsroot on
ROOT.gSystem.Load("/cvmfs/sft.cern.ch/lcg/releases/LCG_85swan3/vdt/0.3.6/x86_64-slc6-gcc49-opt/lib/libvdt.so")
ROOT.gSystem.Load("../../lib/libHiggsAnalysisCombinedLimit.so")

# Setup python 
# import CMS tdrStyle
from tdrStyle import *
setTDRStyle()

# import some more python modules
import sys,glob
from array import array

# Simple Counting Experiment

Lets understand the simplest posssible datacard:

In [ ]:
os.system('cat simple-counting-experiment.txt')

This datacard has just one channel with just one signal and one background, and two nuisance parameters. It is easy to understand how combine reconstructs the likelihood function from this datacard.

The combine tool converts datacards into a RooWorkspace to build the likelihood. You can run "text2workspace.py simple-counting-experiment.txt" and you should end up with a file called simple-counting-experiment.root, or you can use the one which already produced. Lets run this...

In [ ]:
%%bash

source ../../env_standalone.sh
text2workspace.py simple-counting-experiment.txt

## From Datacard to Likelihood

In [ ]:
f_simple = ROOT.TFile("simple-counting-experiment.root","READ")
w_simple = f_simple.Get("w")
w_simple.Print()

By expanding out the "model_s" pdf, it is easy to see that it follows the generic definition of a likelihood function.

# Shape Experiment with Templates

Lets look at a datcard which uses binned histograms for the observable. Each bin of the histogram can just be thought of as a separate channel in a counting experiment.

In [ ]:
os.system('cat simple-shapes-TH1.txt')

There is a new line compared to a counting experiment, one that starts with "shapes". Lets open that file and see what is inside.

In [ ]:
f_input_shapes_TH1 = ROOT.TFile("input-shapes-TH1.root","READ")
f_input_shapes_TH1.ls()

This file contains 1D histograms for the signal, background, and observed data. Also, there are different histograms corresponding to different values of the nuisance parameters. Lets inspect those histograms for the background.

In [ ]:
h_background = f_input_shapes_TH1.Get("background")
h_background.SetLineColor(1)

h_background_alphaUp = f_input_shapes_TH1.Get("background_alphaUp")
h_background_alphaUp.SetLineColor(2)

h_background_alphaDown = f_input_shapes_TH1.Get("background_alphaDown")
h_background_alphaDown.SetLineColor(4)

c = ROOT.TCanvas()
h_background_alphaDown.Draw()
h_background.Draw("same")
h_background_alphaUp.Draw("same")

c.Draw()

As you can see the shape of the histograms are different for the Up/Down variations of the nuisance parameter alpha. Also, the normalization of the histograms are also different, which means this nuisance affects both the shape and rate of the background process:

In [ ]:
print "central = %g, alpha up = %g, alpha down = %g"%(h_background.Integral(), h_background_alphaUp.Integral(), h_background_alphaDown.Integral())

Lets now convert to a RooWorkspace. Again, you can run "text2workspace.py simple-shapes-TH1.txt" in the terminal, or use the already created simple-shapes-TH1.root file. Lets open it and print the contents of the workspace:

In [ ]:
f_shapes_TH1 = ROOT.TFile("simple-shapes-TH1.root","READ")
w_shapes_TH1 = f_shapes_TH1.Get("w")
w_shapes_TH1.Print()

As you can see, the workspace contains "model_s" i.e. the likelihood function, which is composed of the observable "CMS_th1x" which is split in several channels, the POI "r" and several nuisance parameters. Lets see how combine smoothly morphs the shape of the background as a function of the nuisance parameter alpha:

In [ ]:
shapeBkg = w_shapes_TH1.pdf("shapeBkg_bin1_background_morph")
th1x = w_shapes_TH1.var("CMS_th1x")
plot_th1x = th1x.frame()
alpha = w_shapes_TH1.var("alpha")
alpha.setVal(0.0)
shapeBkg.plotOn(plot_th1x,ROOT.RooFit.LineColor(1))
alpha.setVal(0.5)
shapeBkg.plotOn(plot_th1x,ROOT.RooFit.LineColor(3))
alpha.setVal(1.0)
shapeBkg.plotOn(plot_th1x,ROOT.RooFit.LineColor(2))
alpha.setVal(2.0)
shapeBkg.plotOn(plot_th1x,ROOT.RooFit.LineColor(6))

c2 = ROOT.TCanvas()
plot_th1x.Draw()
c2.Draw()

As you can see the shape at alpha=1.0 is the same as the input histogram. In addition there is now a shape for any value of the nuisance parameter alpha.

<b><span style="color:red">Q: What about other systematics in the workspace? Try to plot different values of other nuisances and see how the shapes change) </span></b>

# Parametric Shape Experiment

Lets now look at a shape experiment where the shapes are described by PDF's instead of templates. We will consider a realistic example from the CMS search for H->gamma gamma at 8 TeV. Lets first look at the datacard:

In [ ]:
os.system('cat hgg_8TeV_MVA_cat0145.txt')

As you can see there are a lot of nuisance parameters and channels, typical of a realistic analysis. Lets just focus in on the "shapes" line. You can see the syntax is a bit different. For a parametric shape analysis, the shapes are RooAbsPdf stored in RooWorkspaces. 

Lets look at the background shape. First we can plot the observed data in the "cat0" channel and plot the background only shape.

In [ ]:
f_hgg_bkgdata = ROOT.TFile("hgg.inputbkgdata_8TeV_MVA.root","READ")
w_hgg_bkgdata = f_hgg_bkgdata.Get("cms_hgg_workspace")

CMS_hgg_mass = w_hgg_bkgdata.var("CMS_hgg_mass")
hgg_plot_cat0 = CMS_hgg_mass.frame()

data_cat0 = w_hgg_bkgdata.data("roohist_data_mass_cat0")
data_cat0.plotOn(hgg_plot_cat0)

pdf_bkg_cat0 = w_hgg_bkgdata.pdf("pdf_data_pol_model_8TeV_cat0")
pdf_bkg_cat0.plotOn(hgg_plot_cat0)

c3 = ROOT.TCanvas()
hgg_plot_cat0.Draw()
c3.Draw()

In this datacard the dataset is a binned dataset and the background shape is a smooth pdf. The observed dataset could also be unbinned. 

Lets now look at the signal shapes. First we convert the datacard to a workspace ("text2workspace.py hgg_8TeV_MVA_cat0145.txt"):

In [ ]:
f_hgg = ROOT.TFile("hgg_8TeV_MVA_cat0145.root","READ")
w_hgg = f_hgg.Get("w")
#w_hgg.Print()

You can print the workspace to see what is inside, but it is quite long. If you do, you will see that the shape of the ggH signal in cat0 is called "shapeSig_ggH_cat0". Lets see what parameters it depends on:

In [ ]:
data_obs = w_hgg.data("data_obs")
data_obs.Print()

pdf_ggH = w_hgg.pdf("shapeSig_ggH_cat0")
ggH_params = pdf_ggH.getParameters(data_obs)
ggH_params.Print()

As you can see, the observable is called "CMS_hgg_mass" and the shape of the ggH signal depends on some nuisance parameters as well as MH. Lets draw the ggH PDF for different values of MH and the nuisance parameter "CMS_hgg_globalscale".

In [ ]:

CMS_hgg_mass = w_hgg.var("CMS_hgg_mass")
hgg_plot_sig = CMS_hgg_mass.frame()
w_hgg.var("MH").setVal(122.0)
pdf_ggH.plotOn(hgg_plot_sig,ROOT.RooFit.LineColor(2))
w_hgg.var("MH").setVal(125.0)
pdf_ggH.plotOn(hgg_plot_sig,ROOT.RooFit.LineColor(1))
w_hgg.var("MH").setVal(128.0)
pdf_ggH.plotOn(hgg_plot_sig,ROOT.RooFit.LineColor(4))

w_hgg.var("MH").setVal(125.0)
w_hgg.var("CMS_hgg_globalscale").setVal(0.023585)
pdf_ggH.plotOn(hgg_plot_sig,ROOT.RooFit.LineColor(1),ROOT.RooFit.LineStyle(2))
w_hgg.var("CMS_hgg_globalscale").setVal(-0.023585)
pdf_ggH.plotOn(hgg_plot_sig,ROOT.RooFit.LineColor(1),ROOT.RooFit.LineStyle(3))

c4 = ROOT.TCanvas()
hgg_plot_sig.Draw()
c4.Draw()

As you can see the position of the peak changes as a function of MH, and also a function of the nuisance parameter. In combine the user can encode any dependence of the shape of the PDF's on the model parameters.

Now lets try to draw the combined Signal+Background PDF:

In [ ]:
w_hgg.var("MH").setVal(125.0)
w_hgg.var("CMS_hgg_globalscale").setVal(0.0)

hgg_plot = CMS_hgg_mass.frame()
pdf_bincat0 = w_hgg.pdf("pdf_bincat0")

data_cat0 = data_obs.reduce(ROOT.RooFit.Cut("CMS_channel==CMS_channel::cat0"))
data_cat0.plotOn(hgg_plot)


pdf_bincat0.plotOn(hgg_plot,ROOT.RooFit.ProjWData(data_cat0,True),ROOT.RooFit.LineColor(2))
w_hgg.var("r").setVal(5.0)
pdf_bincat0.plotOn(hgg_plot,ROOT.RooFit.ProjWData(data_cat0,True),ROOT.RooFit.LineColor(3))

c5 = ROOT.TCanvas()
hgg_plot.Draw()
c5.Draw()

As you can see, the total signal+background PDF can now be drawn for any value of the model parameters. In this case we have drawn the total PDF for different values of the signal strength (r=1 and r=5).  

# Asymptotic Limits

Lets go through some of the basic functionality of Combine using the H->gamma gamma workspace. We will start with Asymptotic limits. Lets start by computing the expected and observed limit for m(H) = 125 GeV. You will need to switch to the terminal and run the following command: 

The "-n" gives the output root file a custom name. 
The "-m" sets the value of "MH" in the workspace, and also changes the name of the output root file.
The "-M" option tells combine the stastical method, in this case Asymptotic Limits.
The "hgg_8TeV_MVA_cat0145.root" is the input workspace
The option "--run both" tells combine to run both expected and observed limits. 

You should get an output like this:

In [ ]:
os.system('cat limit125.txt')

You can see we have computed an observed limit on r, and also the nominal expected, and +/- 1 and 2 sigma expected limits. These results are also stored in an output .root file. Lets look at the file and see whats inside:

In [ ]:
f_hgg_limit_125 = ROOT.TFile("higgsCombineLimitTest.Asymptotic.mH125.root")
f_hgg_limit_125.ls()

There is a TTree called limit which keeps the results of the limit computation. Lets look at it:

In [ ]:
limit = f_hgg_limit_125.Get("limit")
for i in xrange(limit.GetEntries()):
    limit.GetEntry(i)
    print i,limit.mh,limit.limit

So you can see that the entries 0-4 are the expected limit and entry 5 is the observed limit. 

We can run similar commands for a range of mass values using a simple script and then make a limit plot from the results. Here is such a script to run the limits:

In [ ]:
os.system('cat run_hgg_asymptotic.sh')

Lets execute this script...

<b><span style="color:red">Q: This should take ~3 minutes and you should end up with a bunch of .root files and .txt files for different masses in a folder called results_hgg_asymptotic. Take a look and see what is there, what do the root files contain? </span></b> 

Here is an example of a simple script to plot the results:

In [ ]:


# create some arrays to hold the results values
mass = array('d',[])
zeros = array('d',[])
exp_p2 = array('d',[])
exp_p1 = array('d',[])
exp = array('d',[])
exp_m1 = array('d',[])
exp_m2 = array('d',[])
obs = array('d',[])

# gather all the results files and sort the mass values
sortedmass = []
files=glob.glob("results_hgg_asymptotic/higgsCombineLimitTest.Asymptotic.mH*.root")
for afile in files:
    m = afile.split('mH')[1].replace('.root','')
    sortedmass.append(float(m))
sortedmass.sort()

# loop over the mass values and fill the arrays
for m in sortedmass:
    # mass value
    mass.append(m)
    # get the limit tree for this mass value
    f = ROOT.TFile("results_hgg_asymptotic/higgsCombineLimitTest.Asymptotic.mH"+str(m).replace('.0','')+".root","READ")
    t = f.Get("limit")
    # expected limit
    t.GetEntry(2)
    thisexp = t.limit
    exp.append(thisexp)
    #-2 sigma
    t.GetEntry(0)
    exp_m2.append(thisexp-t.limit)
    #-1 sigma
    t.GetEntry(1)
    exp_m1.append(thisexp-t.limit)
    #+1 sigma 
    t.GetEntry(3)
    exp_p1.append(t.limit-thisexp)
    #+2 sigma
    t.GetEntry(4)
    exp_p2.append(t.limit-thisexp)
    # observed limit
    t.GetEntry(5)
    obs.append(t.limit)
    # dummy array with 0.0 (for mass-uncertainty)
    zeros.append(0.0)

# convert arrays to TVectorD
v_mass = ROOT.TVectorD(len(mass),mass)
v_zeros = ROOT.TVectorD(len(zeros),zeros)
v_exp_p2 = ROOT.TVectorD(len(exp_p2),exp_p2)
v_exp_p1 = ROOT.TVectorD(len(exp_p1),exp_p1)
v_exp = ROOT.TVectorD(len(exp),exp)
v_exp_m1 = ROOT.TVectorD(len(exp_m1),exp_m1)
v_exp_m2 = ROOT.TVectorD(len(exp_m2),exp_m2)
v_obs = ROOT.TVectorD(len(obs),obs)

#new canvas

c6 = ROOT.TCanvas("c6","c6",800,800)
c6.SetGridx()
c6.SetGridy()
c6.SetRightMargin(0.06)
c6.SetLeftMargin(0.2)

# dummy historgram for axes labels, ranges, etc.
dummy = ROOT.TH1D("","", 1, 115,145)
dummy.SetBinContent(1,0.0)
dummy.GetXaxis().SetTitle('m(H) [GeV]')
dummy.GetYaxis().SetTitle('#sigma / #sigma(SM)')
dummy.SetLineColor(0)
dummy.SetLineWidth(0)
dummy.SetFillColor(0)
dummy.SetMinimum(0.0)
dummy.SetMaximum(5.0)
dummy.Draw()

gr_exp2 = ROOT.TGraphAsymmErrors(v_mass,v_exp,v_zeros,v_zeros,v_exp_m2,v_exp_p2)
gr_exp2.SetLineColor(ROOT.kYellow)
gr_exp2.SetFillColor(ROOT.kYellow)
gr_exp2.Draw("e3same")

gr_exp1 = ROOT.TGraphAsymmErrors(v_mass,v_exp,v_zeros,v_zeros,v_exp_m1,v_exp_p1)
gr_exp1.SetLineColor(ROOT.kGreen)
gr_exp1.SetFillColor(ROOT.kGreen)
gr_exp1.Draw("e3same")

gr_exp = ROOT.TGraphAsymmErrors(v_mass,v_exp,v_zeros,v_zeros,v_zeros,v_zeros)
gr_exp.SetLineColor(1)
gr_exp.SetLineWidth(2)
gr_exp.SetLineStyle(2)
gr_exp.Draw("Lsame")

gr_obs = ROOT.TGraphAsymmErrors(v_mass,v_obs,v_zeros,v_zeros,v_zeros,v_zeros)
gr_obs.SetLineColor(1)
gr_obs.SetLineWidth(2)
gr_obs.Draw("CPsame")

latex2 = ROOT.TLatex()
latex2.SetNDC()
latex2.SetTextSize(0.5*c6.GetTopMargin())
latex2.SetTextFont(42)
latex2.SetTextAlign(31) # align right                                                                                             
latex2.DrawLatex(0.87, 0.95,"19.6 fb^{-1} (8 TeV)")
latex2.SetTextSize(0.9*c6.GetTopMargin())
latex2.SetTextFont(62)
latex2.SetTextAlign(11) # align right                                                                                             
latex2.DrawLatex(0.25, 0.85, "CMS")
latex2.SetTextSize(0.7*c6.GetTopMargin())
latex2.SetTextFont(52)
latex2.SetTextAlign(11)
latex2.DrawLatex(0.25, 0.8, "Tutorial")

legend = ROOT.TLegend(.60,.70,.90,.90)
legend.AddEntry(gr_obs , "Observed 95% CL", "l")
legend.AddEntry(gr_exp , "Expected 95% CL", "l")
legend.AddEntry(gr_exp1 , "#pm 1#sigma", "f")
legend.AddEntry(gr_exp2 , "#pm 2#sigma", "f")
legend.SetShadowColor(0)
legend.SetFillColor(0)
legend.SetLineColor(0)
legend.Draw("same")

ROOT.gPad.RedrawAxis()

c6.Draw()


# Comparing limit Calculations

In addition to the Asymptotic calculations, there are two other ways to calculate the limits

1) CLs using toys 

2) Bayesian limits

For many cases, these two will typically agree, both in fact use the same definition of the likelihood. The treatment of the systematics is subtley different between the two, however, which can lead to differences. In the first, the nuisance parameters are *profiled* in the evaluation of the likelihood while for the second, the nuisances are *marginalised* (integrated over).

Lets see how the two compare for our simple counting experiment analysis

<b>*1) CLs from toys*</b>

The quantity CLs is calculated from the ratio of two p-values.

CLs = CLs+b/CLb 

These are tail probabilities of the distribution of our test statistic under the hypothesis of a signal + background (CLs+b) and the hypothesis of no signal (CLb). We can obtain these distributions by generating toy data.


Lets do this for a single value of the signal multiplier r=1


In [ ]:
%%bash
source ../../env_standalone.sh
combine simple-counting-experiment.root -n LimitToys_r1 -M HybridNew -T 5000 --fullB --singlePoint r=1 --clsAcc 0 --saveHybridResult --freq

Now, lets look at the distributions. There is a script to help you do this 

In [ ]:
%%bash
source ../../env_standalone.sh
python ../../test/plotTestStatCLs.py -i higgsCombineLimitToys_r1.HybridNew.mH120.root -m 120 -v all

In [ ]:
fi_toys  = ROOT.TFile.Open("cls_qmu_distributions.root")
canv = fi_toys.Get("qmu_120.0_1")
canv.Draw()

You might have expected to see Gaussian distributions here but this is a result of our choice of test-statistic (the LHC one). You can try the same but remove the option `--freq` from the HybridNew command


To get a limit on the signal strength, we need to calculate the value of CLs for different values of "r" and find the value for which the value of CLs hits our threshold. For 95% limits, this is 0.05. We'll let combine do this for us by removing the `--singlePoint` command and the `--clsAcc 0`. Removing the last option tells combine to keep throwing toys to reach a certain accuracy on the limit. 

In [ ]:
%%bash
source ../../env_standalone.sh
combine simple-counting-experiment.root -n LimitToys_all -m 125 -M HybridNew --rMax 2 --freq

<b><span style="color:red">Q: What is the result from the Asymptotic calculation? Does it agree?</span></b>

<b>*2) Bayesian Limits*</b>

Next, we can check the result of the Bayesian calculation for the limit. This time, rather than generating toys, the goal is to determine the posterior distribtution of "r" using Bayes' theorem in which the nuisance parameters are marginalised in order to turn the joint pdf of "r" and the nuisances into the posterior on "r". The following will do this for us using a Markov Chain MC integration. 

In [ ]:
%%bash
source ../../env_standalone.sh

combine simple-counting-experiment.root -M MarkovChainMC -n Limit_Bayes --noDefaultPrior 0 --saveChain

We can plot the chains from the output file using a bit of RooFit. 

In [ ]:
fi_ws = ROOT.TFile.Open("simple-counting-experiment.root")
wspace = fi_ws.Get("w")
fi_MCMC = ROOT.TFile.Open("higgsCombineLimit_Bayes.MarkovChainMC.mH120.root")

# Sum up all of the chains / or could take the average limit
mychain=0
for k in fi_MCMC.Get("toys").GetListOfKeys():
    obj = k.ReadObj
    if mychain ==0: 
        mychain = k.ReadObj().GetAsDataSet()
    else :
        mychain.append(k.ReadObj().GetAsDataSet())
        
wspace.var("r").setMax(2)
plot = wspace.var("r").frame()
mychain.plotOn(plot)
c6a = ROOT.TCanvas()
plot.Draw()
c6a.Draw()

<b><span style="color:red">Q: Where does the value for the limit come from? How can the above be turned into a posterior?) </span></b>

<b><span style="color:red">Q: We assumed a *flat* prior on r. What about a prior that 1/sqrt(r) is flat? (add `--prior '1./sqrt(r)'`) </span></b>

# Computing Significance

Now lets compute the expected and observed significance of the signal as a function of m(H). For the expected significance, this can be done for mH = 125 GeV by running the following command:

In [ ]:
%%bash
source ../../env_standalone.sh

combine -n SignifExp -M ProfileLikelihood --signif -m 125 hgg_8TeV_MVA_cat0145.root -t -1 --expectSignal=1 --pvalue > results_hgg_pvalue/expsignif125.txt

You should end up with the following output:

In [ ]:
os.system('cat results_hgg_pvalue/expsignif125.txt')

For the observed, we just remove the "-t -1 --expectSignal=1":

In [ ]:
%%bash
source ../../env_standalone.sh

combine -n SignifExp -M ProfileLikelihood --signif -m 125 hgg_8TeV_MVA_cat0145.root --pvalue > results_hgg_pvalue/signif125.txt

In [ ]:
%%bash 
cat results_hgg_pvalue/obssignif125.txt

So we observe a little less signal than we expected. Lets plot the observed and expected significance as a function of mH. Here is a script to run the command for each mass point:

In [ ]:
%%bash 
cat run_hgg_pvalue.sh

You can run it by doing:

In [ ]:
%%bash
source ../../env_standalone.sh

chmod u+x run_hgg_pvalue.sh
./run_hgg_pvalue.sh

Again you will end up with a bunch of .txt and .root files in the folder results_hgg_pvalue. 

Here is an example script to plot the results:

In [ ]:
unsortedmass = []

mass = array('d',[])
zeros = array('d',[])
exp = array('d',[])
obs = array('d',[])

files=glob.glob("results_hgg_pvalue/higgsCombineSignifExp.ProfileLikelihood.mH*.root")
for afile in files:
    m = afile.split('mH')[1].replace('.root','')
    unsortedmass.append(float(m))
unsortedmass.sort()

for m in unsortedmass:
    # mass value
    mass.append(m)
    # get the expected p-value
    f_exp = ROOT.TFile("results_hgg_pvalue/higgsCombineSignifExp.ProfileLikelihood.mH"+str(m).replace('.0','')+".root","READ")
    t_exp = f_exp.Get("limit")
    t_exp.GetEntry(0)
    exp.append(t_exp.limit)
    # get the observed p-value
    f_obs = ROOT.TFile("results_hgg_pvalue/higgsCombineSignifObs.ProfileLikelihood.mH"+str(m).replace('.0','')+".root","READ")
    t_obs = f_obs.Get("limit")
    t_obs.GetEntry(0)
    obs.append(t_obs.limit)
    # dummy, for mass error
    zeros.append(0.0)

# convert array to TVector
v_mass = ROOT.TVectorD(len(mass),mass)
v_zeros = ROOT.TVectorD(len(zeros),zeros)
v_exp = ROOT.TVectorD(len(exp),exp)
v_obs = ROOT.TVectorD(len(obs),obs)
# new canvas
c7 = ROOT.TCanvas("c7","c7",800, 800)
c7.SetLogy()
c7.SetRightMargin(0.06)
c7.SetLeftMargin(0.2)
# dummy histogram, for axis labels, ranges, etc.
dummy = ROOT.TH1D("","", 1, 115,145)
dummy.SetBinContent(1,0.0)
dummy.GetXaxis().SetTitle('m(H) [GeV]')
dummy.GetYaxis().SetTitle('Local p-value')
dummy.SetLineColor(0)
dummy.SetLineWidth(0)
dummy.SetFillColor(0)
dummy.SetMinimum(0.0001)
dummy.SetMaximum(1.0)
dummy.Draw()

# Draw some lines corresponding to 1,2,3 sigma 
latexf = ROOT.TLatex()
latexf.SetTextSize(0.4*c7.GetTopMargin())
latexf.SetTextColor(2)
f1 = ROOT.TF1("f1","0.15866",115,145)
f1.SetLineColor(2)
f1.SetLineWidth(2)
f1.Draw("lsame")
latexf.DrawLatex(116, 0.15866*1.1,"1#sigma")
f2 = ROOT.TF1("f1","0.02275",115,145)
f2.SetLineColor(2)
f2.SetLineWidth(2)
f2.Draw("lsame")
latexf.DrawLatex(116, 0.02275*1.1,"2#sigma")
f3 = ROOT.TF1("f1","0.0013499",115,145)
f3.SetLineColor(2)
f3.SetLineWidth(2)
f3.Draw("lsame")
latexf.DrawLatex(116, 0.0013499*1.1,"3#sigma")

# Draw the expected p-value graph
gr_exp = ROOT.TGraphAsymmErrors(v_mass,v_exp,v_zeros,v_zeros,v_zeros,v_zeros)
gr_exp.SetLineColor(4)
gr_exp.SetLineWidth(2)
gr_exp.SetLineStyle(2)
gr_exp.Draw("Lsame")
# Draw the observed p-value graph
gr_obs = ROOT.TGraphAsymmErrors(v_mass,v_obs,v_zeros,v_zeros,v_zeros,v_zeros)
gr_obs.SetLineColor(1)
gr_obs.SetLineWidth(2)
gr_obs.Draw("CPsame")

latex2 = ROOT.TLatex()
latex2.SetNDC()
latex2.SetTextSize(0.5*c7.GetTopMargin())
latex2.SetTextFont(42)
latex2.SetTextAlign(31) # align right                                                                                                                              
latex2.DrawLatex(0.87, 0.95,"19.6 fb^{-1} (8 TeV)")
latex2.SetTextSize(0.7*c7.GetTopMargin())
latex2.SetTextFont(62)
latex2.SetTextAlign(11) # align right                                                                                                                              
latex2.DrawLatex(0.20, 0.95, "CMS")
latex2.SetTextSize(0.6*c7.GetTopMargin())
latex2.SetTextFont(52)
latex2.SetTextAlign(11)
latex2.DrawLatex(0.32, 0.95, "Tutorial")

c7.Draw()

# Maximum Likelihood Fits

Suppose now that we want to measure the signal strength at M(H)=125 GeV. We can use the MaxLikelihoodFit and MultiDimFit (maximum likelihood fit for an arbitrary number of POIs) methods.

We can get the best fit for the signal strength by running this command:

In [ ]:
%%bash
source ../../env_standalone.sh

combine -M MaxLikelihoodFit -m 125 hgg_8TeV_MVA_cat0145.root

We can also create likelihood scans by manually defining a range for the POI and computing the deltaNLL at each point. Lets do it for both the expected and observed:

In [ ]:
%%bash
source ../../env_standalone.sh

combine -n Obs -M MultiDimFit -m 125 hgg_8TeV_MVA_cat0145.root --algo=grid --points 15 --setPhysicsModelParameterRanges r=0.0,3.0

In [ ]:
%%bash
source ../../env_standalone.sh

combine -n Exp -M MultiDimFit -m 125 hgg_8TeV_MVA_cat0145.root --algo=grid --points 15 --setPhysicsModelParameterRanges r=0.0,3.0 -t -1 --expectSignal=1 

Here is an example script to plot the scans:

In [ ]:
# create arrays

r_exp = array('d',[])
nll_exp = array('d',[])
r_obs = array('d',[])
nll_obs = array('d',[])
zeros = array('d',[])
# get expected scan
f_exp = ROOT.TFile("higgsCombineExp.MultiDimFit.mH125.root","READ")
t_exp = f_exp.Get("limit")
for i in xrange(1,t_exp.GetEntries()):
    t_exp.GetEntry(i)
    r_exp.append(t_exp.r)
    nll_exp.append(2.0*t_exp.deltaNLL)
# get observed scan
f_obs = ROOT.TFile("higgsCombineObs.MultiDimFit.mH125.root","READ")
t_obs = f_obs.Get("limit")
for i in xrange(1,t_obs.GetEntries()):
    t_obs.GetEntry(i)
    r_obs.append(t_obs.r)
    nll_obs.append(2.0*t_obs.deltaNLL)
    zeros.append(0.0)
# convert arrays to TVectorD
v_r_exp = ROOT.TVectorD(len(r_exp),r_exp)
v_r_obs = ROOT.TVectorD(len(r_obs),r_obs)
v_nll_exp = ROOT.TVectorD(len(nll_exp),nll_exp)
v_nll_obs = ROOT.TVectorD(len(nll_obs),nll_obs)
v_zeros = ROOT.TVectorD(len(zeros),zeros)
# new canvas
c9 = ROOT.TCanvas("c9","c9",800, 800)
c9.SetRightMargin(0.06)
c9.SetLeftMargin(0.2)
# dummy for axis labels, ranges, etc.
dummy = ROOT.TH1D("","", 1, 0.0,3.0)
dummy.SetBinContent(1,0.0)
dummy.GetXaxis().SetTitle('#sigma/#sigma_{SM}')
dummy.GetYaxis().SetTitle('-2 #Delta lnL')
dummy.SetLineColor(0)
dummy.SetLineWidth(0)
dummy.SetFillColor(0)
dummy.SetMinimum(0.0)
dummy.SetMaximum(5.0)
dummy.Draw()
# Draw some lines for 68% CL and 95% CL
latexf = ROOT.TLatex()
latexf.SetTextSize(0.4*c9.GetTopMargin())
latexf.SetTextColor(2)
f1 = ROOT.TF1("f1","1.0",0.0,3.0)
f1.SetLineColor(2)
f1.SetLineWidth(2)
f1.Draw("lsame")
latexf.DrawLatex(2.5, 1.1,"68% CL")
f2 = ROOT.TF1("f1","3.84",0.0,3.0)
f2.SetLineColor(2)
f2.SetLineWidth(2)
f2.Draw("lsame")
latexf.DrawLatex(2.5, 3.94,"95% CL")
# draw expected scan
gr_exp = ROOT.TGraphAsymmErrors(v_r_exp,v_nll_exp,v_zeros,v_zeros,v_zeros,v_zeros)
gr_exp.SetLineColor(1)
gr_exp.SetLineWidth(2)
gr_exp.SetLineStyle(2)
gr_exp.Draw("Lsame")
# draw observed scan
gr_obs = ROOT.TGraphAsymmErrors(v_r_obs,v_nll_obs,v_zeros,v_zeros,v_zeros,v_zeros)
gr_obs.SetLineColor(1)
gr_obs.SetLineColor(1)
gr_obs.SetLineWidth(2)
gr_obs.Draw("Lsame")

latex2 = ROOT.TLatex()
latex2.SetNDC()
latex2.SetTextSize(0.5*c9.GetTopMargin())
latex2.SetTextFont(42)
latex2.SetTextAlign(31) # align right                                                                                                                              
latex2.DrawLatex(0.87, 0.95,"19.6 fb^{-1} (8 TeV)")
latex2.SetTextSize(0.7*c9.GetTopMargin())
latex2.SetTextFont(62)
latex2.SetTextAlign(11) # align right                                                                                                                              
latex2.DrawLatex(0.20, 0.95, "CMS")
latex2.SetTextSize(0.6*c9.GetTopMargin())
latex2.SetTextFont(52)
latex2.SetTextAlign(11)
latex2.DrawLatex(0.32, 0.95, "Tutorial")

legend = ROOT.TLegend(.60,.14,.90,.26)
legend.AddEntry(gr_obs , "Observed", "l")
legend.AddEntry(gr_exp , "Expected", "l")
legend.SetShadowColor(0)
legend.SetFillColor(0)
legend.SetLineColor(0)
legend.Draw("same")

ROOT.gPad.RedrawAxis()

c9.Draw()


<b><span style="color:red">Q: What would the uncertainty in my signal strengh be in this case?</span></b>

#  MultiDimensional Scans

Here we will create a 2D confidence interval for kV vs kF.

kV scales the diagrams which involve a Higgs-vector boson coupling while kF scales the diagrams which involve a Higgs-fermion coupling (which ones are they?)

first we build the model ... 

In [ ]:
%%bash
source ../../env_standalone.sh

text2workspace.py hgg_8TeV_MVA_cat0145.txt -m 125.7  -P HiggsAnalysis.CombinedLimit.PhysicsModel:rVrFXSHiggs  -o rVrF_hgg.root

and then we can run the fit using that model. We tell combine which ranges to scan over and how many points to run in the scan

In [ ]:
%%bash
source ../../env_standalone.sh

combine -M MultiDimFit rVrF_hgg.root -m 125 -n HggHttCvCf --algo=grid --points=100 --setPhysicsModelParameterRanges RF=-1,3:RV=-2,5

Now lets make a plot of the result of the scan. The output tree contains the grid of points and the value of the log-likelihood, relative to that at the best fit point. This is what we can use to define 1 and 2 sigma contours. 

In [ ]:
file_mdf = ROOT.TFile.Open("higgsCombineHggHttCvCf.MultiDimFit.mH125.root")
limit = file_mdf.Get("limit")
c10 = ROOT.TCanvas()
limit.Draw("RV:RF>>h2","2*deltaNLL*(deltaNLL >= 0)","COL");
c10.Draw()

<b><span style="color:red">Q: Now add the 1/2-sigma contours to the plot. </span></b>


# Channel Compatibility

Lets now compute the best fit signal strength in each category, but with full correlation of all the nuisance parameters. In order to do that, switch to the terminal and run the following command:

In [ ]:
%%bash
source ../../env_standalone.sh

combine -M ChannelCompatibilityCheck hgg_8TeV_MVA_cat0145.root -m 125 --saveFitResult > ccc.txt

You should get an output like this:

In [ ]:
os.system('cat ccc.txt')

Here is a script to plot the results:

In [ ]:
import ROOT

poi = "r"; rMax = 4
infile="higgsCombineTest.ChannelCompatibilityCheck.mH125.root"

filein = ROOT.TFile(infile,"READ")
fit_nominal = filein.Get("fit_nominal");
fit_alternate = filein.Get("fit_alternate");
rFit = fit_nominal.floatParsFinal().find(poi);

prefix = "_ChannelCompatibilityCheck_"+poi+"_"

nChann = 0;
iter = fit_alternate.floatParsFinal().createIterator();

while True:
  a = iter.Next();
  if a==None: break
  if (prefix in a.GetName()): nChann+=1

frame = ROOT.TH2F("frame",";best fit #sigma/#sigma(SM);",1,rFit.getMin(),min(rFit.getMax(),rMax),nChann,0,nChann)

iter.Reset();
iChann = 0;
points = ROOT.TGraphAsymmErrors(nChann)

while True:
  a = iter.Next();
  if a==None: break
  if (prefix in a.GetName()):
    channel = a.GetName();
    channel.replace(prefix,"")
    points.SetPoint(iChann, a.getVal(), iChann+0.5);
    points.SetPointError(iChann, -a.getAsymErrorLo(), a.getAsymErrorHi(), 0, 0);
    iChann+=1;
    frame.GetYaxis().SetBinLabel(iChann, channel.replace(prefix,""));

c8 = ROOT.TCanvas("c8","c8",800,800)
c8.cd()                                                                                                                                
c8.SetTopMargin(0.07)                                                                                                                  
c8.SetBottomMargin(0.12)                                                                                                               
c8.SetLeftMargin(0.12)                                                                                                                 
                                                                                                                                      
points.SetLineColor(ROOT.kRed);                                                                                                       
points.SetLineWidth(3);                                                                                                               
points.SetMarkerStyle(21);                                                                                                            
frame.GetXaxis().SetTitleSize(0.05);                                                                                                  
frame.GetXaxis().SetLabelSize(0.04);                                                                                                  
frame.GetYaxis().SetLabelSize(0.06);                                                                                                  
frame.Draw();                                                                                                                         
ROOT.gStyle.SetOptStat(0);                                                                                                            
globalFitBand = ROOT.TBox(rFit.getVal()+rFit.getAsymErrorLo(), 0, rFit.getVal()+rFit.getAsymErrorHi(), nChann);                       
globalFitBand.SetFillColor(65);     
globalFitBand.SetFillStyle(3003);                                                                                                       
globalFitBand.SetLineStyle(0);                                                                                                        
globalFitBand.Draw("SAME");                                                                                                           
globalFitLine = ROOT.TLine(rFit.getVal(), 0, rFit.getVal(), nChann);                                                                  
globalFitLine.SetLineWidth(4);                                                                                                        
globalFitLine.SetLineColor(214);                                                                                                      
globalFitLine.Draw("SAME");                                                                                                           
points.Draw("P");                                                                                                                
                                                                                                                                      
latex2 = ROOT.TLatex()                                                                                                                
latex2.SetNDC()                                                                                                                       
latex2.SetTextSize(0.5*c8.GetTopMargin())                                                                                              
latex2.SetTextFont(42)                                                                                                                
latex2.SetTextAlign(31) # align right                                                                                                 
latex2.DrawLatex(0.87, 0.95,"19.6 fb^{-1} (8 TeV)")                                                                                   
latex2.SetTextSize(0.7*c8.GetTopMargin())                                                                                              
latex2.SetTextFont(62)                                                                                                                
latex2.SetTextAlign(11) # align right                                                                                                 
latex2.DrawLatex(0.15, 0.95, "CMS")                                                                                                   
latex2.SetTextSize(0.6*c8.GetTopMargin())                                                                                              
latex2.SetTextFont(52)                                                                                                                
latex2.SetTextAlign(11)                                                                                                               
latex2.DrawLatex(0.27, 0.95, "Tutorial")                                                                                              
                                                                                                                                      
ROOT.gPad.RedrawAxis()    
c8.Update()
c8.Draw()                  

<b><span style="color:red">Q: Do you think the different measurements are compatible? How could we quantify how well this result agrees with what we might expect? </span></b>